## Step 1: 환경 설정 및 의존성 설치

In [1]:
# 필수 패키지 설치
!pip install -q transformers einops huggingface_hub matplotlib scikit-learn biopython torch pandas numpy

## Step 2: 모듈 임포트

In [2]:
import sys
import os
import gc
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import torch

# 현재 디렉토리를 경로에 추가 (모듈 로드용)
current_dir = Path('.').resolve()
project_root = current_dir.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 모듈 임포트
from preparation import get_device, load_models
from sequence_generation import (
    fetch_gene_sequences, sort_genes_by_length, 
    DEFAULT_DECODING_STRATEGIES
)

print("✅ 모든 모듈 임포트 완료")

✅ 모든 모듈 임포트 완료


## Step 3: 디바이스 초기화 및 모델 로드

In [3]:
# 디바이스 감지 (CUDA / MPS / CPU)
device = get_device()
print(f"\n사용 중인 디바이스: {device}")

PyTorch Version: 2.9.1
Using device: mps

사용 중인 디바이스: mps


In [4]:
# 모델 로드
model_configs = {
    "DNABERT-2": "zhihan1996/DNABERT-2-117M",
    "NT-v2-50m": "InstaDeepAI/nucleotide-transformer-v2-50m-multi-species",
    "NT-v2-500m": "InstaDeepAI/nucleotide-transformer-v2-500m-multi-species",
}

models = load_models(device, model_configs=model_configs)
print(f"로드된 모델: {list(models.keys())}")


📥 Downloading DNABERT-2...


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

✅ DNABERT-2 Triton patch applied successfully.
[DNABERT-2] Loading model...


/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(


[DNABERT-2] Model loaded successfully.
✅ DNABERT-2 loaded successfully.
[NT-v2-50m] Loading model...
[NT-v2-50m] Model loaded successfully.
✅ NT-v2-50m loaded successfully.
[NT-v2-500m] Loading model...
[NT-v2-500m] Model loaded successfully.
✅ NT-v2-500m loaded successfully.

🚀 3 model(s) ready!
로드된 모델: ['DNABERT-2', 'NT-v2-50m', 'NT-v2-500m']


## Step 4: 유전자 시퀀스 수집 및 생성

In [5]:
# 설정
NCBI_EMAIL = "your_email@example.com"
ITERATIONS = 30  # 진화 스텝 수
MASK_RATIO = 0.2  # 마스킹 비율
RESULTS_DIR = Path('results/sequences')
RESULTS_DIR.mkdir(exist_ok=True)

print(f"설정:")
print(f"  - 반복 횟수: {ITERATIONS}")
print(f"  - 마스킹 비율: {MASK_RATIO}")
print(f"  - 결과 디렉토리: {RESULTS_DIR}")

설정:
  - 반복 횟수: 30
  - 마스킹 비율: 0.2
  - 결과 디렉토리: results/sequences


In [6]:
# 유전자 시퀀스 수집
gene_selection = fetch_gene_sequences(NCBI_EMAIL)
gene_selection = sort_genes_by_length(gene_selection)

print("\n수집된 유전자:")
for gene, seq in gene_selection.items():
    print(f"  - {gene}: {len(seq)} bp")

Fetching gene sequences from NCBI...
Fetching H4C1 by UID NM_003538...
  ✅ Found and added sequence for H4C1 (Length: 402bp)
Fetching TP53 by UID NM_000546...
  ✅ Found and added sequence for TP53 (Length: 2512bp)
Fetching GAPDH by UID NM_002046...
  ✅ Found and added sequence for GAPDH (Length: 1285bp)
Fetching GAPDHP1 by UID NG_001123...
  ✅ Found and added sequence for GAPDHP1 (Length: 1098bp)
Fetching NORAD by UID NR_027451...
  ✅ Found and added sequence for NORAD (Length: 5378bp)
Fetching STAT3 by UID NM_139276...
  ✅ Found and added sequence for STAT3 (Length: 4921bp)
Fetching TTN by UID NM_001267550...
  ✅ Found and added sequence for TTN (Length: 109224bp)
Fetching HBB by UID NM_000518...
  ✅ Found and added sequence for HBB (Length: 628bp)
Fetching HOXC11 by UID NM_014212...
  ✅ Found and added sequence for HOXC11 (Length: 3261bp)
Fetching HOTAIR by UID NR_003716...
  ✅ Found and added sequence for HOTAIR (Length: 2273bp)
Fetching VEGFA by UID NM_003376...
  ✅ Found and added

In [7]:
# 토큰 길이 테이블 (모델별 / 유전자별)
token_length_rows = []

for model_label, model_instance in models.items():
    tokenizer = model_instance.tokenizer
    for gene_id, sequence in gene_selection.items():
        enc = tokenizer(sequence, add_special_tokens=True, truncation=False)
        token_len = len(enc["input_ids"])
        avg_token_bp = len(sequence) / token_len if token_len else float('nan')
        token_length_rows.append({
            "gene": gene_id,
            "model": model_label,
            "token_len": token_len,
            "avg_token_bp": avg_token_bp,
        })

token_length_df = (
    pd.DataFrame(token_length_rows)
    .pivot(index="gene", columns="model", values="token_len")
    .sort_index()
)

avg_token_bp_df = (
    pd.DataFrame(token_length_rows)
    .pivot(index="gene", columns="model", values="avg_token_bp")
    .sort_index()
)

# 모델별 평균 (유전자 평균)
token_length_df.loc["MEAN"] = token_length_df.mean(axis=0)
avg_token_bp_df.loc["MEAN"] = avg_token_bp_df.mean(axis=0)

# 1024 초과 강조 스타일
def highlight_over_limit(value, limit=1024):
    if pd.isna(value):
        return ''
    return 'color: #b91c1c; font-weight: 600;' if value > limit else ''

print("토큰 개수")
display(token_length_df.style.applymap(highlight_over_limit))

print("한 토큰 평균 길이(bp)")
display(avg_token_bp_df)


Token indices sequence length is longer than the specified maximum sequence length for this model (18205 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (18205 > 2048). Running this sequence through the model will result in indexing errors


토큰 개수


/var/folders/wm/m60tntks4_55jf79w27yb1t40000gn/T/ipykernel_54248/2178913799.py:40: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  display(token_length_df.style.applymap(highlight_over_limit))


model,DNABERT-2,NT-v2-500m,NT-v2-50m
gene,,,
GAPDH,265.000000,216.000000,216.000000
GAPDHP1,222.000000,184.000000,184.000000
H4C1,91.000000,68.000000,68.000000
HBB,127.000000,109.000000,109.000000
HOTAIR,464.000000,384.000000,384.000000
HOXC11,648.000000,547.000000,547.000000
NEAT1,641.000000,536.000000,536.000000
NORAD,1069.000000,899.000000,899.000000
PTEN,1663.000000,1421.000000,1421.000000


한 토큰 평균 길이(bp)


model,DNABERT-2,NT-v2-500m,NT-v2-50m
gene,,,
GAPDH,4.849057,5.949074,5.949074
GAPDHP1,4.945946,5.967391,5.967391
H4C1,4.417582,5.911765,5.911765
HBB,4.944882,5.761468,5.761468
HOTAIR,4.898707,5.919271,5.919271
HOXC11,5.032407,5.961609,5.961609
NEAT1,4.976599,5.951493,5.951493
NORAD,5.030870,5.982202,5.982202
PTEN,5.120265,5.992259,5.992259


In [7]:
# 시퀀스 생성 및 CSV 저장
print("\n시퀀스 생성 중...\n" + "="*70)

for model_label, model_instance in models.items():
    print(f"\n모델: {model_label}")
    
    for gene_id, original_sequence in gene_selection.items():
        print(f"  {gene_id}...", end="", flush=True)
        
        # 출력 경로
        model_name = model_label.replace("/", "-")
        model_dir = RESULTS_DIR / model_name
        model_dir.mkdir(parents=True, exist_ok=True)
        output_csv = model_dir / f"{gene_id}.csv"
        
        # 결과 저장용 데이터프레임
        results_data = {}
        
        for strategy_base_key, strategy_cfg in DEFAULT_DECODING_STRATEGIES.items():
            strategy_type = strategy_cfg["type"]
            temperatures = strategy_cfg.get("temperatures", [1.0])
            top_k = strategy_cfg.get("top_k", 50)
            
            for temp in temperatures:
                # 전략 이름
                if strategy_type == "greedy":
                    strategy_key = strategy_base_key
                else:
                    strategy_key = f"{strategy_base_key}_t{temp}"
                
                # 시퀀스 생성
                generated_sequences = model_instance.run(
                    sequence=original_sequence,
                    steps=ITERATIONS,
                    mask_ratio=MASK_RATIO,
                    strategy=strategy_type,
                    temperature=temp,
                    top_k=top_k,
                    save_all=True,
                    save_interval=1
                )
                
                # 결과 저장 (각 iteration을 column으로)
                results_data[strategy_key] = generated_sequences
                
                # 메모리 정리
                del generated_sequences
                gc.collect()
                if device == "cuda":
                    torch.cuda.empty_cache()
                elif device == "mps":
                    torch.mps.empty_cache()
        
        # DataFrame 생성 (decoding strategy를 row로)
        df = pd.DataFrame(results_data).T
        
        # 열 이름을 iteration 번호로 설정
        df.columns = [f"iteration_{i}" for i in range(df.shape[1])]
        
        # CSV 저장
        df.to_csv(output_csv)
        
        print(f" ✓ ({len(df)} strategies)")

print("\n" + "="*70)
print("✅ 시퀀스 생성 완료!")
print(f"결과 저장 위치: {RESULTS_DIR}")

# 기존 임베딩 캐시 정리 (새 시퀀스 기준으로 다시 계산 필요)
embeddings_dir = Path('results/embeddings')
if embeddings_dir.exists():
    removed = 0
    for cache_file in embeddings_dir.glob('**/*.pkl'):
        try:
            cache_file.unlink()
            removed += 1
        except Exception as e:
            print(f'  캐시 삭제 실패: {cache_file} ({e})')
    print(f'🧹 기존 임베딩 캐시 삭제 완료: {removed} files')
else:
    print('🧹 기존 임베딩 캐시 없음')



시퀀스 생성 중...

모델: DNABERT-2
  H4C1... ✓ (4 strategies)
  HBB... ✓ (4 strategies)
  GAPDHP1... ✓ (4 strategies)
  GAPDH... ✓ (4 strategies)
  TPI1... ✓ (4 strategies)
  TPI1P1... ✓ (4 strategies)
  HOTAIR...

/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 512 to 516
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 516 to 519
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 519 to 520
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 520 to 521
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 521 to 525
  warnings.warn(
/Users/leeminjae/.cache/h

 ✓ (4 strategies)
  TP53...

/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 544 to 546
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 546 to 553
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 553 to 557
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 557 to 558
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 558 to 566
  warnings.warn(
/Users/leeminjae/.cache/h

 ✓ (4 strategies)
  NEAT1...

...

 ✓ (4 strategies)
  PTENP1...

/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 835 to 839
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 839 to 845
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 845 to 848
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 848 to 852
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 852 to 861
  warnings.warn(
/Users/leeminjae/.cache/h

 ✓ (4 strategies)
  STAT3...

/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 866 to 983
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 983 to 988
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 988 to 989
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 989 to 1000
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 1000 to 1008
  warnings.warn(
/Users/leeminjae/.cach

 ✓ (4 strategies)
  NORAD... ✓ (4 strategies)
  PTEN... ✓ (4 strategies)
  TTN... ✓ (4 strategies)

모델: NT-v2-50m
  H4C1... ✓ (4 strategies)
  HBB... ✓ (4 strategies)
  GAPDHP1... ✓ (4 strategies)
  GAPDH... ✓ (4 strategies)
  TPI1... ✓ (4 strategies)
  TPI1P1... ✓ (4 strategies)
  HOTAIR... ✓ (4 strategies)
  TP53... ✓ (4 strategies)
  NEAT1... ✓ (4 strategies)
  HOXC11... ✓ (4 strategies)
  VEGFA... ✓ (4 strategies)
  PTENP1... ✓ (4 strategies)
  STAT3... ✓ (4 strategies)
  NORAD... ✓ (4 strategies)
  PTEN... ✓ (4 strategies)
  TTN... ✓ (4 strategies)

모델: NT-v2-500m
  H4C1... ✓ (4 strategies)
  HBB... ✓ (4 strategies)
  GAPDHP1... ✓ (4 strategies)
  GAPDH... ✓ (4 strategies)
  TPI1... ✓ (4 strategies)
  TPI1P1... ✓ (4 strategies)
  HOTAIR... ✓ (4 strategies)
  TP53... ✓ (4 strategies)
  NEAT1... ✓ (4 strategies)
  HOXC11... ✓ (4 strategies)
  VEGFA... ✓ (4 strategies)
  PTENP1... ✓ (4 strategies)
  STAT3... ✓ (4 strategies)
  NORAD... ✓ (4 strategies)
  PTEN... ✓ (4 strategies)
  TT

## 결과 확인

In [8]:
import pandas as pd

print("생성된 결과 확인:\n")
print("="*70)

for model_key in models.keys():
    model_name = model_key.replace("/", "-")
    model_dir = RESULTS_DIR / model_name
    
    print(f"\n{model_key}:")
    
    for gene_id in gene_selection.keys():
        csv_file = model_dir / f"{gene_id}.csv"
        if csv_file.exists():
            df = pd.read_csv(csv_file, index_col=0)
            num_strategies = len(df)
            num_iterations = len(df.columns)
            print(f"  {gene_id:10s}: {num_strategies:2d} strategies, {num_iterations:2d} iterations")
            if num_strategies > 0:
                print(f"            First strategy: {df.index[0]}")
        else:
            print(f"  {gene_id:10s}: ✗ FILE NOT FOUND")

print("\n" + "="*70)

생성된 결과 확인:


DNABERT-2:
  H4C1      :  4 strategies, 31 iterations
            First strategy: greedy
  HBB       :  4 strategies, 31 iterations
            First strategy: greedy
  GAPDHP1   :  4 strategies, 31 iterations
            First strategy: greedy
  GAPDH     :  4 strategies, 31 iterations
            First strategy: greedy
  TPI1      :  4 strategies, 31 iterations
            First strategy: greedy
  TPI1P1    :  4 strategies, 31 iterations
            First strategy: greedy
  HOTAIR    :  4 strategies, 31 iterations
            First strategy: greedy
  TP53      :  4 strategies, 31 iterations
            First strategy: greedy
  NEAT1     :  4 strategies, 31 iterations
            First strategy: greedy
  HOXC11    :  4 strategies, 31 iterations
            First strategy: greedy
  VEGFA     :  4 strategies, 31 iterations
            First strategy: greedy
  PTENP1    :  4 strategies, 31 iterations
            First strategy: greedy
  STAT3     :  4 strategies, 31 iteratio